In [43]:
import os
import sys
from pathlib import Path
import importlib
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [44]:
import src.domlm as model 
from transformers import AutoModel, DataCollatorForLanguageModeling
importlib.reload(model)

<module 'src.domlm' from '/Users/ilyalasy/dev/DOM-LM/src/domlm/__init__.py'>

In [45]:
roberta = AutoModel.from_pretrained("roberta-base")
roberta_config = roberta.config

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
roberta_config_dict = roberta_config.to_dict()
roberta_config_dict["_name_or_path"] = "domlm"
roberta_config_dict["architectures"] = ["DOMLMForMaskedLM"]
domlm_config = model.DOMLMConfig.from_dict(roberta_config_dict)
domlm_config.save_pretrained("../domlm-config/")
domlm = model.DOMLMForMaskedLM(domlm_config)

In [37]:
from collections import OrderedDict

state_dict = OrderedDict((f"domlm.{k}",v) for k,v in roberta.state_dict().items())
domlm.load_state_dict(state_dict,strict=False)

_IncompatibleKeys(missing_keys=['domlm.embeddings.tree_embeddings.node_embeddings.weight', 'domlm.embeddings.tree_embeddings.parent_embeddings.weight', 'domlm.embeddings.tree_embeddings.sibling_embeddings.weight', 'domlm.embeddings.tree_embeddings.depth_embeddings.weight', 'domlm.embeddings.tree_embeddings.tag_embeddings.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'], unexpected_keys=['domlm.pooler.dense.weight', 'domlm.pooler.dense.bias'])

In [47]:
import src.preprocess as preprocess 
importlib.reload(preprocess)

base_path = Path("experiments/data/Calendar")

with open(base_path / 'CalendarSelect_1.html','r') as f:
    html = f.read()
subtrees = preprocess.extract_features(html,domlm_config)

In [48]:
tokenizer = preprocess.tokenizer

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)